In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Import packages and change directory

In [2]:
import sys

sys.path.insert(0, "/content/drive/MyDrive/Colab Notebooks/SOC_Profile_Generation")
%cd "/content/drive/MyDrive/Colab Notebooks/SOC_Profile_Generation"

/content/drive/MyDrive/Colab Notebooks/SOC_Profile_Generation


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import csv
import os.path

from household import Household
from rank_households import rank_households
from create_profiles import create_profiles

### Load data

In [4]:
### Mobility Data
# priority aggregation method: always mobility acitivity
# no priority: always longest activity
mobility_data_pkl_path = '/content/drive/MyDrive/Tobias_MA/Data/data_mop_priority.pkl'
#mobility_data_pkl_path = '/content/drive/MyDrive/Tobias_MA/Data/data_mop.pkl'


### TANK Data (from MOP):
# needed for car segment (in column [174])
# Household_ID in column [0]
mob_car_info_csv_path = '/content/drive/MyDrive/Tobias_MA/Data/TANK18.csv'


### Electric Car database for car substitution
# [:,3]: Battery capacity [kWh]
# [:,4]: WLTP-consumption [kWh / 100km]
# [:,5]: Car Charging power [kW]
el_cars_csv_path = '/content/drive/MyDrive/Tobias_MA/Data/Elektroauto_Datenbank.csv'


### Temperature [°C] in time period (same timesteps)
weather_csv_path = '/content/drive/MyDrive/Tobias_MA/Data/Temperaturen_Deutschland_2017.csv'


### Timesteps
# Input timestep length and number of timesteps

ts_length = 10
no_of_ts = 1008
#no_of_ts = int (states_mop.shape[1])   # needs to process data first

In [5]:
### Data Processing

# pkl file
with open(mobility_data_pkl_path, 'rb') as input:
        [states_mop,speed_mop,meta_mop,meta_header_mop] = pickle.load(input)
# observe only year 2017
meta_mop_data = meta_mop[-3820:-1,]
states_mop_data = states_mop[-3820:-1,]
speed_mop_data = speed_mop[-3820:-1,]

# csv files
mob_car_info_csv = np.genfromtxt (mob_car_info_csv_path, 
                                  delimiter=";", 
                                  encoding = "ISO-8859-1")

el_cars_csv = np.genfromtxt (el_cars_csv_path, 
                             delimiter=";", 
                             encoding = "ISO-8859-1")

weather_csv = np.genfromtxt (weather_csv_path, 
                             delimiter=";", 
                             encoding = "ISO-8859-1")

### Rank households

In [12]:
# Rank households according to your input

# Choose the following main parameters (will be fulfilled)
input_number_of_occupants = 1
input_number_of_drivers = 1
input_number_of_cars = 1


# Choose the following five soft parameters and their weight.
# Fitting households will be sorted by weighted soft parameters.
input_income = 7
input_w_income = 0.2

input_population = 2
input_w_population = 0.2

input_year_of_birth = 1980
input_w_year_of_birth = 0.2

input_job = 1
input_w_job = 0.2

input_distance = 10000
input_w_distance = 0.2

# How many households should be returned?
# Put "all", if all fitting households should be returned
quantity = 10

###

households = rank_households(meta_mop_data,
                             states_mop_data,
                             speed_mop_data,
                             no_of_ts, 
                             ts_length,
                             input_number_of_occupants,
                             input_number_of_drivers,
                             input_number_of_cars, 
                             input_income,
                             input_w_income,
                             input_population,
                             input_w_population,
                             input_year_of_birth,
                             input_w_year_of_birth,
                             input_job,
                             input_w_job,
                             input_distance,
                             input_w_distance,
                             quantity)
print(households)

/content/drive/MyDrive/Colab Notebooks/SOC_Profile_Generation/rank_households.py:157: RuntimeWarning: overflow encountered in long_scalars
  job_diff = job_hh - job
/content/drive/MyDrive/Colab Notebooks/SOC_Profile_Generation/rank_households.py:124: RuntimeWarning: overflow encountered in long_scalars
  income_diff = income_hh - income


Number of fitting hosueholds: 366
[4701034955, 4511021288, 4501025405, 4501023822, 4301013963, 4711030459, 4511021849, 4511020331, 4501024210, 4501026011]


### SOC Profile Generation

In [ ]:
### Create profiles for each car from every household

'''
Choose the following parameters:
- household_list:           put "households" for whole list from rank_households
- start, end:               first and last timestep
- home_charging_power:      possible charging power at home [kW]
- work_charging_power:      -"- at work -> only one of both can be 0
- charging_efficiency:      efficiency while charging
- discharging_efficiency:   efficiency while discharging (= consuming)
- min_charge:               SOC can not go lower (relative to battery capacity)
- max_charge:               SOC can not go higher (relative to battery capacity)
- path:                     path where csv-outputs should be saved
- bool_plot:                true, if plots should be created
- bool_create_csv:          true, if csv-files should be created
'''

household_list = households
start = 0
end = 1008
home_charging_power = 11
work_charging_power = 5
charging_efficiency = 0.95
discharging_efficiency = 0.95
min_charge = 10  # [%]
max_charge = 90  # [%]
path = '/content/drive/MyDrive/Tobias_MA/Outputs/'
bool_plot = True
bool_create_csv = False

###

create_profiles(household_list,
                meta_mop_data,
                states_mop_data,
                speed_mop_data,
                start,
                end,
                no_of_ts,
                ts_length,
                home_charging_power,
                work_charging_power,
                charging_efficiency,
                discharging_efficiency,
                min_charge,
                max_charge,
                weather_csv,
                mob_car_info_csv,
                el_cars_csv,
                path,
                bool_plot,
                bool_create_csv)